<a href="https://colab.research.google.com/github/yoojinlee-hub/Recommended-skills_Study_by_python/blob/main/2_3_Measuring_the_accuracy_of_recommender_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd

base_src = 'drive/MyDrive/practice_recommanding'
u_user_src = os.path.join(base_src,'u.user')
u_cols=['user_id','age','sex','occupation','zip_code']

users = pd.read_csv(u_user_src,sep='|',names=u_cols,encoding='latin-1')
users = users.set_index('user_id')

u_item_src=os.path.join(base_src,'u.item')
i_cols=['movie_id','title','release date','video release date','IMDB URL','unknown','Action','Adventuser','Animation','Children\'s','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']

movies = pd.read_csv(u_item_src,sep='|',names=i_cols,encoding='latin-1')
movies = movies.set_index('movie_id')

u_data_src=os.path.join(base_src,'u.data')
r_cols=['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(u_data_src, sep='\t',names=r_cols,encoding='latin-1')
ratings=ratings.set_index('user_id')



---



In [3]:
# 100X의 영화 평점에 대해서 실제값과 best-seller 방식으로 구한 예측값의 RMSE를 계산
import numpy as np

def RMSE(y_true,y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

# 정확도 계산
rmse = []
movie_mean = ratings.groupby(['movie_id'])['rating'].mean()

for user in set(ratings.index):
  y_true = ratings.loc[user]['rating']
  # best-seller 방식
  y_pred = movie_mean[ratings.loc[user]['movie_id']]
  accuracy = RMSE(y_true,y_pred)
  rmse.append(accuracy)

# RMSE 계산
print(np.mean(rmse))

0.996007224010567




---

사용자 집단별 추천

In [10]:
import os
import pandas as pd

base_src = 'drive/MyDrive/practice_recommanding'
u_user_src = os.path.join(base_src,'u.user')
u_cols=['user_id','age','sex','occupation','zip_code']

users = pd.read_csv(u_user_src,sep='|',names=u_cols,encoding='latin-1')
users = users.set_index('user_id')

u_item_src=os.path.join(base_src,'u.item')
i_cols=['movie_id','title','release date','video release date','IMDB URL','unknown','Action','Adventuser','Animation','Children\'s','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']

movies = pd.read_csv(u_item_src,sep='|',names=i_cols,encoding='latin-1')
movies = movies.set_index('movie_id')

u_data_src=os.path.join(base_src,'u.data')
r_cols=['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(u_data_src, sep='\t',names=r_cols,encoding='latin-1')

#ratings DataFrame 에서 timestamp 제거
#ratings = ratings.drop('timestamp',axis=1)
#movies = movies[['movie_id','title']]

In [13]:
#데이터 train, test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']

x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.25,stratify=y)

#정확도를 계산하는 함수
def RMSE(y_true,y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

#모델별 RMSE를 계산하는 함수
def score(model):
  id_pairs = zip(x_test['user_id'],x_test['movie_id'])
  y_pred = np.array([model(user,movie) for (user,movie) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true,y_pred)

#best seller 함수를 이용한 정확도 계산
train_mean = x_train.groupby(['movie_id'])['rating'].mean()
def best_seller(user_id,movie_id):
  try:
    rating = train_mean[movie_id]

  except:
    rating = 3.0
  return rating

score(best_seller)

1.028183679795263

In [17]:
#성별에 따른 예측값 계산
merged_ratings = pd.merge(x_train,users) ## ERROR

users = users.set_index('user_id')

g_mean = merged_ratings[['movie_id','sex','rating']].groupby(['movie_id','sex'])['rating'].mean()
g_mean

MergeError: ignored

In [20]:
rating_matrix = x_train.pivot(index='user_id',columns='movie_id',values='rating')
rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1669,1670,1673,1674,1675,1676,1677,1678,1680,1681
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
#gender 기준 추천
def cf_gender(user_id,movie_id):
  if movie_id in rating_matrix.columns:
    gender = users.loc[user_id]['sex']
    if gender in g_mean[movie_id].index: ##ERROR
      gender_rating = g_mean[movie_id][gender]
    else:
      gender_ratings = 3.0
  else:
    gender_rating = 3.0
  return gender_ratings

score(cf_gender)

NameError: ignored